# simpletransformers
使用pip install simpletransformers命令
使用之前，最好更新一下transformers库，因为这个simple版本相当于给transformers库进行再封装。
### 使得高级预训练模型（BERT、RoBERTa、XLNet、XLM、DistilBERT、ALBERT、CamemBERT、XLM-RoBERTa、FlauBERT）的训练、评估和预测变得简单，每条只需3行即可初始化模型。当前支持序列分类（二进制，多类，多标签，句子对），序列标注（NER）、问答、回归、会话式AI和多模态任务。该库主要基于Hugging Face 的Transformer库。

这里的数据集在可以下载到：https://www.kesci.com/home/dataset/5e7a0d6398d4a8002d2cd201/files

预训练模型在：

https://huggingface.co/models

In [77]:
from simpletransformers.classification import ClassificationModel
import pandas as pd
import smart_open
import numpy as np 
from sklearn import preprocessing

In [78]:
#载入数据
data1 = open(r'data\muti_class1776\train_data.word.ed',encoding='utf-8').readlines()
data2 = open(r'data\muti_class1776\val_data.word.ed',encoding='utf-8').readlines()
# len(data1),len(data2)
data1

['society\\t陆家嘴 不 雅 视频 曝光 ： 系恶 “ 性 ” 营销 ？\n',
 'world\\t日本 冲绳 民众 举行 大规模 抗美 集会 活动 冲绳 县知事 : 希望 改变 政治 结构\n',
 'entertainment\\t林心如 全黑 连体 装 现身 ， 被 赞比 模特 身材 更好\n',
 'car\\t别克 逆 生长 实验 型\n',
 'baby\\t你 的 宝宝 到底 需不需要 补锌 ？ 又 该 如何 补充 ？ （ 内附 补锌 食谱 ）\n',
 'sports\\t名记 为 快船 支招 ： 格里芬 换 小将 再 签 KD\n',
 'tech\\t装逼 课堂 丨 如何 快速 加微信 好友 加 到 爆机\n',
 'society\\t山西 ： 安全带 保 安全 （ 一 ） — — 不系 安全带 事故 酿 大祸\n',
 'military\\t陆空 对抗 ， 友邻 互为 “ 磨刀石 ”\n',
 'entertainment\\ttvN 推 全新 综艺 《 Babel250 》 外国人 组团 创新 语言\n',
 'entertainment\\t“ 中国 男团 ” 征战 欧洲 秀场 胡歌 爆发 时尚 力\n',
 'tech\\t微软 梁戈 碧 ： 微软 用 实际行动 爱 Linux\n',
 'sports\\t英格兰 的 上限 有 多 高 ？ 真核 浮现 最大 X 因素 就是 他\n',
 'finance\\t同洲 电子 ： 袁明 辞职 承诺 6 个 月 内 不 办理 股权 转让\n',
 'story\\t讽刺 故事 ： 喷嚏\n',
 'society\\t中环 高架桥 路面 受损 肇事 车辆 行驶 路径 已 查明\n',
 'sports\\t宁泽涛 参加 里 约 奥运 峰回路转\n',
 'car\\t百万 宝马 + 001 牛牌 被 弃 停车场 ！ 想 去 开 回来 却 交不起 停车费 ！\n',
 'game\\t娱乐 卡组 推荐 ： 小明术\n',
 'society\\t先 结婚 再 办房 房子 凭 啥 不是 夫妻 共有 财产 ？\n',
 'world\\t俄媒 : 美越 关系 亲近 或 损害 俄中 在 东南亚 的 利益\n',
 'car\\t交通部 要 使出 绝杀 了 ！ Out 吧 ， 飞机 板 ！

In [79]:
# 对载入数据进行预处理
# 分割类别标签和预测文本
labels1,labels2,text1,text2=[],[],[],[]
split_data1=[i.split(r'\t') for i in data1]
for i in split_data1:
    if len(i)==2:
        labels1.append(i[0])
        text1.append(i[1])
# labels1=[i[0] for i in split_data1]
# text1=[i[1] for i in split_data1]

split_data2=[i.split(r'\t') for i in data2]
for i in split_data2:
    if len(i)==2:
        labels2.append(i[0])
        text2.append(i[1])
# labels2=[i[0] for i in split_data2]
# text2=[i[1] for i in split_data2]
len(labels1),len(text1)

(334492, 334492)

In [80]:
# 对格式化的数据进行表格化
# 用pandas创建一个两列的表格，第一列为类别标签，第二列为文本，默认无表头。
train_df=pd.DataFrame([i,j] for i,j in zip(labels1,text1))
eval_df=pd.DataFrame([i,j] for i,j in zip(labels2,text2))

In [81]:
# 检视数据。
train_df[0][:10]

0          society
1            world
2    entertainment
3              car
4             baby
5           sports
6             tech
7          society
8         military
9    entertainment
Name: 0, dtype: object

In [82]:
# 标签数值化
# sklearn.preprocessing.LabelEncoder()：标准化标签，将标签值统一转换成range(标签值个数-1)范围内。
le=preprocessing.LabelEncoder()
le.fit(np.unique(train_df[0].tolist()))
print('标签值标准化:%s' % le.transform(["world", "entertainment", "car", "baby","entertainment"]))
print('标准化标签值反转:%s' % le.inverse_transform([0, 2 ,0 ,1 ,2]))

标签值标准化:[17  3  1  0  3]
标准化标签值反转:['baby' 'discovery' 'baby' 'car' 'discovery']


In [83]:
#
train_df[2] = train_df[0].apply(lambda x:le.transform([x])[0])
eval_df[2] = eval_df[0].apply(lambda x:le.transform([x])[0])

del train_df[0]
del eval_df[0]

In [84]:
# 统计不重复的标签数量。
num_labels=len(np.unique(train_df[2].tolist()))
num_labels

18

In [85]:
# 保存预处理后的训练数据。只有前面加上r之后才能保存到内嵌文件夹
train_df.to_csv(r'data\muti_class1776\train_data.csv',header=False,sep='\t',index = False)
eval_df.to_csv(r'data\muti_class1776\test_data.csv',header=False,sep='\t',index = False)

In [86]:
# 检视预处理后的训练数据。
train_df.head()

,1,2
0,陆家嘴 不 雅 视频 曝光 ： 系恶 “ 性 ” 营销 ？\n,12
1,日本 冲绳 民众 举行 大规模 抗美 集会 活动 冲绳 县知事 : 希望 改变 政治 结构\n,17
2,林心如 全黑 连体 装 现身 ， 被 赞比 模特 身材 更好\n,3
3,别克 逆 生长 实验 型\n,1
4,你 的 宝宝 到底 需不需要 补锌 ？ 又 该 如何 补充 ？ （ 内附 补锌 食谱 ）\n,0


In [87]:
# 检视预处理后的验证数据。
eval_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47784 entries, 0 to 47783
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   1       47784 non-null  object
 1   2       47784 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 746.8+ KB


# 构建分配模型
### 从应用落地的角度来说，bert虽然效果好，但有一个短板就是预训练模型太大，预测时间在平均在300ms以上（一条数据），无法满足业务需求。知识蒸馏是在较低成本下有效提升预测速度的方法。
### 在不丧失精度和预测效果的前提下，为了提高预测时的速度，以及节省空间，此处载入的是BERT的“蒸馏版” --- distibert，这是一种能够将大型BERT模型（被称为「老师」）压缩为较小模型（即「学生」）的技术。
## 原理：
### - 知识蒸馏：迁移泛化能力
### 知识蒸馏（有时也称为师生学习）是一种压缩技术，要求对小型模型进行训练，以使其拥有类似于大型模型（或者模型集合）的行为特征。这项技术由 Bucila 等人提出，并得到了 Hinton 等人的推广。我们这里采用的，正是 Hinton 采取的方法。
### 在监督学习领域，我们在训练分类模型时往往会利用对数似然信号实现概率最大化（logits 的 softmax），进而预测出正确类。在大多数情况下，性能良好的模型能够利用具有高概率的正确类预测输出分布，同时其它类的发生概率则接近于零。
### 但是，某些“接近于零”的概率要比其它概率更大，这在一定程度上反映出模型的泛化能力。
### 例如，把普通椅子误认为扶手椅虽然属于错误，但这种错误远比将其误认为蘑菇来得轻微。这种不确定性，有时被称为“暗知识”。
### 我们也可以从另一个角度来理解蒸馏——用于防止模型对预测结果太过确定（类似于标签平滑）。

这里使用的类是ClassificationModel，说明如下：
* ClassificationModel*
- class simpletransformers.classification.ClassificationModel (model_type, model_name, args=None, use_cuda=True)  该类用于文本分类任务。
* Class attributes*
- tokenizer: 使用的预训练tokenizer
- model: 使用的预训练模型（BERT、RoBERTa、XLNet、XLM、DistilBERT、ALBERT等）
- model_name: 预训练模型的名称或者本地模型所在路径（其中的模型文件必须为pytorch_nodel.bin）
- device: 用于训练和评价的设备，cpu、gpu（cuda）或者tpu。
- results: 模型的评价结果，以dictionary的形式存在
- args: 用于training和evaluation的参数dict，形如args={'reprocess_input_data': True, 'overwrite_output_dir': True})
- cuda_device(可选) : 默认为 -1. 用于指定使用哪个GPU
* Parameters*
- model_type: (必须) str - 使用的预训练模型的类型，当前支持BERT、RoBERTa、XLNet、XLM、DistilBERT、ALBERT、CamemBERT、XLM-RoBERTa、FlauBERT
### model_name: (必须) str - 预训练模型的名称或者本地模型所在路径（其中的模型文件必须为pytorch_nodel.bin）
- num_labels (可选): 数据集中类别的数量
- weight (可选):长度为num_labels（标签数量）的列表，其中包含要分配给每个标签以进行损耗计算的权重数值。
- args: (可选) dict - 参数字典，形如args={'reprocess_input_data': True, 'overwrite_output_dir': True})
- use_cuda: (可选) bool - 默认为 -1. 用于指定使用哪个GPU class methods
- train_model(self, train_df, output_dir=None, show_running_loss=True, args=None, eval_df=None)
* Returns*
- result: 以Dictionary的形式呈现evaluation results. (即Matthews correlation coefficient, tp, tn, fp, fn)
- model_outputs: eval_df中每一行的模型输出列表
- wrong_preds: 与模型的每个错误预测相对应的InputExample对象列表

In [109]:
# 对于多标签分类, 需要将标签的类别数传递给num_labels这个参数 ，这里是18个类，故num_labels = 18
# num_labels=18
# model = ClassificationModel(
#     model_type='distilbert',
#     model_name=r"C:/Users/Eric/Desktop/github实现有趣的代码/已完成/Bert-Chinese-Text-Classification-Pytorch/bert_pretrain/pytorch_model.bin",
#     num_labels=num_labels, 
#     args={"reprocess_input_data":True,"overwrite_output_dir":True},# 对输入数据进行预处理# 可覆盖输出文件夹  
#     use_cuda=True,
#     )
model = ClassificationModel(
    "distilbert", 
    r"C:/Users/Eric/Desktop/github实现有趣的代码/已完成/Bert-Chinese-Text-Classification-Pytorch/bert_pretrain/pytorch_model.bin",
    num_labels=num_labels, 
    args={"reprocess_input_data": True,   # 对输入数据进行预处理
          "overwrite_output_dir": True}   # 可覆盖输出文件夹
    )

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x80 in position 0: invalid start byte